In [1]:
import numpy as np
from scipy.stats import levy_stable
from scipy.stats import uniform
from scipy.stats import multivariate_normal

In [2]:
def prior_sample():
    # Sample from prior distribution
    # For univariate alpha-stable models, sample each parameter from its respective uniform distribution
    #np.random.seed(seed)
    alpha = uniform.rvs(1.1, 0.9)  # U[1.1, 2]
    beta = uniform.rvs(-1, 2)  # U[-1, 1]
    gamma = uniform.rvs(0, 300)  # U[0, 300]
    delta = uniform.rvs(-300, 600)  # U[-300, 300]
    return [alpha, beta, gamma, delta]

In [3]:
def univ_alpha_stable_sampler(params,size,seed) :
    alpha,beta,gamma,delta = params[0],params[1],params[2],params[3]
    #print(alpha,beta,gamma,delta)
    y_bar = 0
    np.random.seed(seed)
    w = np.random.standard_exponential(size=size)
    u = np.random.uniform(low = -np.pi/2, high = np.pi/2, size = size)
    if alpha == 1 :
        y_bar = 2/np.pi*((np.pi/2+beta*u)*np.tan(u)-beta*np.log((np.pi/2*w*np.cos(u))/(np.pi/2+beta*u)))
        return gamma*y_bar + delta
    else :
        S = (1+beta**2*np.tan(np.pi*alpha/2)**2)**(1/(2*alpha))
        B = 1/alpha*np.arctan(beta*np.tan(np.pi*alpha/2))
        #print(S,B)
        y_bar = (S*(np.sin(alpha)*(u+B))*(np.cos(u-alpha*(u+B))/w)**((1-alpha)/alpha))/np.cos(u)**(1/alpha)
        return gamma*y_bar + delta

In [4]:
def compute_quantiles(data):
    # Compute quantile-based summary statistics
    quantiles = np.percentile(data, [5, 25, 50, 75, 95])
    v_alpha = (quantiles[4] - quantiles[0]) / (quantiles[3] - quantiles[1])
    v_beta = (quantiles[4] + quantiles[0] - 2 * quantiles[2]) / (quantiles[4] - quantiles[0])
    v_gamma = (quantiles[3] - quantiles[1]) / quantiles[2]
    return v_alpha, v_beta, v_gamma

In [5]:
def S(data):
    # Compute summary statistics of the data
    # This function should return low-dimensional summary statistics S(data)
    # For univariate alpha-stable models, use quantile-based estimators along with the mean of the data
    v_alpha, v_beta, v_gamma = compute_quantiles(data)
    mean_x = np.mean(data)
    return np.array([v_alpha, v_beta, v_gamma, mean_x])

In [6]:
def psi(y, x, covariance = np.diag([0.25,0.25,1,1])):
    # Define smoothing kernel using a Gaussian kernel with estimated covariance
    return multivariate_normal.pdf(y, mean=x, cov=covariance)


In [7]:
def cov_estimate(theta_hat,n_draws = 1000):
    x = np.array([univ_alpha_stable_sampler(params = theta_hat, size = 200, seed=None) for _ in range(n_draws)])
    sumary_statistics = np.array([S(x_i) for x_i in x])

    return np.cov(sumary_statistics, rowvar=False)

In [8]:
def norm2(u) :
    return np.sqrt(np.sum(u*u))


In [9]:
def K(u, sigma) :
    d = sigma.shape[0]
    #print(d)
    return np.exp(-(u.T@np.linalg.inv(sigma)@u)/2)/(np.sqrt(2*np.pi)**d*np.sqrt(np.linalg.det(sigma)))




In [10]:
def resample(theta, weights,N) :
    normalized_weights = weigths/np.sum(weights)
    theta['t'] = np.random.choice(theta['t-1'], size=N, replace=True, p=normalized_weights)
    weights['t'] = np.ones(N)/N

In [11]:
def sample_from_M_t(N, theta_prev, weights_prev):
    normalized_weights = weights_prev/np.sum(weights_prev)
    # Choose a component based on the weights
    component = np.random.choice(range(N), p=normalized_weights)
    # Generate a sample from the chosen component
    sample = np.random.multivariate_normal(mean = theta_prev[component], cov = np.diag([0.25,0.25,1,1]))
    #samples[i] = sample
    return sample 

In [12]:
def M_t(N,theta_i,theta_prev, weights_prev) :
    densities = np.array([ psi(theta_i, theta_prev[i]) for i in range(N)])
    return np.sum(weights_prev*densities)
    

In [13]:
def normalize(u) :
    return u/np.sum(u)


In [33]:
def SMC_sampler_v2(arg, T=10,N =1000,n=200,covariance = np.diag([0.25, 0.25, 1, 1]), seed=42) :
    #epsilons = [k for k in range(1000,100, -100)]+[k for k in range(100,9, -1)] + [k+0.5 for k in range(9,4,-1)] + [5-k*0.05 for k in range(40)]+[3-0.01*k for k in range(301)]  # Tolerance schedule
    epsilons = [100*k for k in range(10,0,-1)]
    T = len(epsilons)

    #y = levy_stable.rvs(1.7, 0.9, loc=10, scale=10, size=200)
    ###--------Initialization :
    epsilon = epsilons[0]
    # 1 - sample theta from the prior : 
    # theta is a Nx4 matrix containing all the N theta_i
    #initial_theta = np.array([prior_sample(seed) for _ in range(N)])
    #theta = {"t" : initial_theta , "t-1" :initial_theta }
    # set weights their initial values :
    #x = np.array([univ_alpha_stable_sampler(params=theta['t'][i], size=n, seed=seed) for i in range(N)])
    #print('x.shape = ',x.shape)
    #initial_weights = np.array([K((S(y)-S(x))/epsilon,sigma_hat(theta_hat))/epsilon for theta_hat in theta['t']])
    #print("weights = ", initial_weights)
    #print(" sum(weights) = ", np.sum(initial_weights))
    #weights = {"t" : initial_weights, "t-1" : initial_weights}
    y,x,theta,weights,sigma_hat = arg
    t=1
    res = []
    
    print("INITIALIZATION IS DONE :)")
    while t<T :
        print("t = ",t)
        epsilon = epsilons[t]
        print("epsilon = ",epsilon)
        accepted = []
        rejected = []
        theta_maj = theta['t']
        weights_maj = weights['t']
        i = 0 
        while i<N : 
            print('###########    i = ',i)
            weights['t'] = normalize(weights['t'])
            weights['t-1'] = normalize(weights['t-1'])
            # Mutation and correction : 
            theta['t'][i] = sample_from_M_t(N, theta['t-1'], weights['t-1'])
            print(f'theta["t"][{i}]= ', np.round(theta['t'][i],3))
            #sigma_hat = cov_estimate(np.array([1.7,0.9,10,10]))
            weights['t'][i] = K((S(y)-S(x[i]))/epsilon,sigma_hat)/epsilon/M_t(N,theta['t'][i],theta['t-1'],weights['t-1'])
            weights['t'] = normalize(weights['t'])
            weights['t-1'] = normalize(weights['t-1'])
            print('weights["t"] = ', np.round(weights['t'][:5],3))
            #theta['t-1'] = theta_maj
            #weights['t-1'] = weights_maj
            u = np.random.uniform(low=0, high=1, size=1)
            c_t = np.quantile(weights['t'], 0.9)
            proba = 1-np.minimum(1,weights['t']/c_t)
            print("c_t = ", c_t)
            print(f"weights['t'][{i}]/c_t = ", weights['t'][i]/c_t)
            print(f'proba[{i}] = ', proba[i])
            if u<=proba[i] :
                print(f"theta['t'][{i}] = {np.round(theta['t'][i],3)} rejected !")
                continue
            else : 
                print(f"theta['t'][{i}] = {np.round(theta['t'][i],3)} accepted !")
                weights['t'][i] = weights['t'][i]/(1-proba[i])
                i = i+1
            """
            rejected = theta['t'][u<=proba]
            accepted = theta['t'][u>proba]
            
            print("accepted = ",accepted)
            print("acceptance pourcentage = ", len(accepted)*100/N)
            print("rejected = " ,rejected)
            print("rejected pourcentage = ", len(rejected)*100/N)
            """
        t+=1
        theta['t-1'],weigths['t-1'] = theta_maj,weights_maj
        res.append((theta['t'], weights['t']))
        # Resample : 
        resample(theta,weights,N)
        print("theta = ", theta[:5])
        print("weights = ", weights[:5])


In [34]:
def initialization(T=10,N =1000,n=200,covariance = np.diag([0.25, 0.25, 1, 1]), seed=42) :
    #epsilons = [k for k in range(1000,100, -100)]+[k for k in range(100,9, -1)] + [k+0.5 for k in range(9,4,-1)] + [5-k*0.05 for k in range(40)]+[3-0.01*k for k in range(301)]  # Tolerance schedule
    epsilons = [100*k for k in range(10,0,-1)]
    T = len(epsilons)
    y = levy_stable.rvs(1.7, 0.9, loc=10, scale=10, size=200)
    ###--------Initialization :
    epsilon = epsilons[0]

    sigma_hat = cov_estimate(np.array([1.7,0.9,10,10]))
    print("sigma_hat = ", sigma_hat)
    print('det(sigma_hat) = ', np.linalg.det(sigma_hat))
    # 1 - sample theta from the prior : 
    # theta is a Nx4 matrix containing all the N theta_i
    initial_theta = np.array([prior_sample() for _ in range(N)])
    theta = {"t" : initial_theta , "t-1" :initial_theta }
    print('theta["t"] = ',np.round(theta['t'][:5],4))
    # set weights their initial values :
    x = np.array([univ_alpha_stable_sampler(params=theta['t'][i], size=n, seed=seed) for i in range(N)])
    print('x.shape = ',x.shape)
    initial_weights = np.array([K((S(y)-S(x[i]))/epsilon,sigma_hat)/epsilon for i in range(N)])
    print("weights = ", initial_weights)
    print(" sum(weights) = ", np.sum(normalize(initial_weights)))
    weights = {"t" : normalize(initial_weights), "t-1" : normalize(initial_weights)}

    return y,x,theta,weights,sigma_hat

In [35]:
arg = initialization()

sigma_hat =  [[ 2.55918415e+00 -1.19911015e-01 -1.33362328e-01 -6.14651504e+00]
 [-1.19911015e-01  2.29877333e-02 -2.21646563e-02  9.36720838e-01]
 [-1.33362328e-01 -2.21646563e-02  3.05373819e-01 -1.74186713e+00]
 [-6.14651504e+00  9.36720838e-01 -1.74186713e+00  2.29235689e+02]]
det(sigma_hat) =  2.0860507099524606
theta["t"] =  [[ 1.451300e+00 -1.288000e-01  4.550100e+01 -2.468921e+02]
 [ 1.711500e+00  4.507000e-01  1.663000e+02 -3.952590e+01]
 [ 1.402700e+00  5.532000e-01  2.489396e+02  1.520650e+01]
 [ 1.402600e+00  1.084000e-01  2.707098e+02  3.476200e+00]
 [ 1.295700e+00  4.390000e-01  2.966141e+02 -1.629173e+02]]
x.shape =  (1000, 200)
weights =  [1.75343911e-005 1.75317799e-005 1.75236172e-005 1.74117153e-005
 1.75050127e-005 1.74994738e-005 1.75371600e-005 1.74426658e-005
 1.75362901e-005 1.75338659e-005 1.75374126e-005 1.75330068e-005
 1.75348189e-005 1.75168665e-005 1.75140988e-005 1.75323692e-005
 1.75373596e-005 1.75115544e-005 1.75357973e-005 1.75370543e-005
 1.74824763e

In [32]:
[100*k for k in range(10,0,-1)]

[1000, 900, 800, 700, 600, 500, 400, 300, 200, 100]

In [36]:
SMC_sampler_v2(arg=arg)

INITIALIZATION IS DONE :)
t =  1
epsilon =  900
###########    i =  0
theta["t"][0]=  [  1.412   0.202  67.237 192.717]
weights["t"] =  [0.123 0.001 0.001 0.001 0.001]
c_t =  0.0008816856248271652
weights['t'][0]/c_t =  140.00689696144198
proba[0] =  0.0
theta['t'][0] = [  1.412   0.202  67.237 192.717] accepted !
###########    i =  1
theta["t"][1]=  [  2.683  -0.36  170.373 149.309]
weights["t"] =  [0.105 0.151 0.001 0.001 0.001]
c_t =  0.0007487705322192822
weights['t'][1]/c_t =  202.33155605678544
proba[1] =  0.0
theta['t'][1] = [  2.683  -0.36  170.373 149.309] accepted !
###########    i =  2
theta["t"][2]=  [  1.742   0.262  40.429 247.11 ]
weights["t"] =  [0.09  0.13  0.144 0.001 0.001]
c_t =  0.000641621833969527
weights['t'][2]/c_t =  224.02779133904812
proba[2] =  0.0
theta['t'][2] = [  1.742   0.262  40.429 247.11 ] accepted !
###########    i =  3
theta["t"][3]=  [   0.19     0.374   10.247 -120.289]
weights["t"] =  [0.076 0.109 0.121 0.158 0.001]
c_t =  0.0005405622990896

KeyboardInterrupt: 